#AI Lab 07
K213309 | BCS-6F

##Task 1

In [4]:
'''Write Python Program for each task, create sample space and calculate the
probability and show the output.
A manufacturing firm employs three analytical plans for the design and development of
a particular product. For cost reasons, all three are used at varying times. In fact, plans
1, 2, and 3 are used for 30%, 20%, and 50% of the products, respectively. The defect
rate is different for the three procedures as follows: P(D|P1)=0.01, P(D|P2)=0.03,
P(D|P3)=0.02, where P(D|Pj ) is the probability of a defective product, given plan j. If a
random product was observed and found to be defective, which plan was most likely
used and thus responsible?'''

import numpy as np

PlanProbs = [0.3,0.2,0.5]
DefectProbs = [0.01,0.03,0.02]

DefectProbs_np = np.array(DefectProbs)
PlanProbs_np = np.array(PlanProbs)

DefectChange = DefectProbs_np * PlanProbs_np
deno = DefectChange.sum()

bayesianPossibility = PlanProbs_np * DefectProbs_np
bayesianPossibility2 = bayesianPossibility/deno
print(bayesianPossibility2)

planNumb = bayesianPossibility2.argmax()
print('With prob of ', bayesianPossibility2[planNumb], 'The plan with defect is', planNumb+1)

[0.15789474 0.31578947 0.52631579]
With prob of  0.5263157894736842 The plan with defect is 3


##Task 2

In [5]:
!pip install pomegranate==v0.14.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 18.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for pomegranate: filename=pomegranate-0.14.9-cp310-cp310-linux_x86_64.whl size=18331536 sha256=78d36d2710f912451874b328617a626b465a82f2f7a3211310cb8629deb52bb5
  Stored in directory: /root/.cache/pip/wheels/14/e7/b2/189a2d351ac4ae073cfa17ce9d56936d59af5712a18028fc31
Successfully built pomegranate
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10


In [19]:
'''Sarah is a plant manager at a manufacturing facility. She&#39;s concerned about two
potential issues that could affect production: a malfunction in one of the machines or a
power outage in the area. Sarah relies on two employees, Alex and Emily, to notify her
if either event occurs. Alex is diligent and always calls Sarah when he notices a
machine malfunction, but occasionally he confuses the sound of the machine with other
noises in the factory and calls mistakenly. Emily, on the other hand, tends to get
engrossed in her work and sometimes misses the sound of the machine
malfunctioning. Given this scenario, we want to calculate the probability of a Machine
Malfunction Alarm.'''

from pomegranate import *

Malfunction = DiscreteDistribution({'True': 0.001, 'False': 0.999})
PowerOutage = DiscreteDistribution({'True': 0.002, 'False': 0.998})
Alarm = ConditionalProbabilityTable(
    [['True', 'True', 'True', 0.95],
     ['True', 'True', 'False', 0.05],
     ['True', 'False', 'True', 0.94],
     ['True', 'False', 'False', 0.06],
     ['False', 'True', 'True', 0.29],
     ['False', 'True', 'False', 0.71],
     ['False', 'False', 'True', 0.001],
     ['False', 'False', 'False', 0.999]],
    [Malfunction, PowerOutage]
)
Alex_calls = ConditionalProbabilityTable(
    [['True', 'True', 0.9],
     ['True', 'False', 0.1],
     ['False', 'True', 0.05],
     ['False', 'False', 0.95]],
    [Alarm]
)
Emily_calls = ConditionalProbabilityTable(
    [['True', 'True', 0.7],
     ['True', 'False', 0.3],
     ['False', 'True', 0.01],
     ['False', 'False', 0.99]],
    [Alarm]
)

s1 = State(Malfunction, name="Malfunction")
s2 = State(PowerOutage, name="PowerOutage")
s3 = State(Alarm, name="Alarm")
s4 = State(Alex_calls, name="Alex_calls")
s5 = State(Emily_calls, name="Emily_calls")

network = BayesianNetwork("Malfunction Alarm")
network.add_states(s1, s2, s3, s4, s5)
network.add_edge(s1, s3)
network.add_edge(s2, s3)
network.add_edge(s3, s4)
network.add_edge(s3, s5)
network.bake()

prob_alarm_given_malfunction = network.predict_proba({'Malfunction': 'False'})[2].parameters[0]['True']
print("Probability of Alarm given Malfunction:", prob_alarm_given_malfunction)


Probability of Alarm given Malfunction: 0.0015780000000007283


##Task 3

In [1]:
'''Suppose there are three boxes, each containing two coins. Box 1 has two
gold coins, Box 2 has two silver coins, and Box 3 has one gold coin and one silver coin.
You randomly pick a box and take out one coin. It turns out to be a gold coin. What is the
probability that the other coin in the box is also gold if:

(a) You picked Box 1?
(b) You picked Box 2?
(c) You picked Box 3?

Use ‘pomegranate’ and create conditional probability table for each box.'''

from pomegranate import *

dist_box1 = DiscreteDistribution({'gold': 1, 'silver': 0})
dist_box2 = DiscreteDistribution({'gold': 0, 'silver': 1})
dist_box3 = DiscreteDistribution({'gold': 0.5, 'silver': 0.5})

box1 = State(dist_box1, name='Box1')
box2 = State(dist_box2, name='Box2')
box3 = State(dist_box3, name='Box3')

model = BayesianNetwork('Coin Boxes')
model.add_states(box1, box2, box3)

model.add_edge(box1, box2)
model.add_edge(box1, box3)
model.add_edge(box2, box1)
model.add_edge(box2, box3)
model.add_edge(box3, box1)
model.add_edge(box3, box2)
model.bake()

print("Conditional Probability Table:")
print(model.states)
for state in model.states:
    if state.name.startswith('Box'):
        print(f"P(Gold | {state.name}):", state.distribution.parameters[0]['gold'])


Conditional Probability Table:
[{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "gold" : 1,
                "silver" : 0
            }
        ],
        "frozen" : false
    },
    "name" : "Box1",
    "weight" : 1.0
}, {
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "gold" : 0,
                "silver" : 1
            }
        ],
        "frozen" : false
    },
    "name" : "Box2",
    "weight" : 1.0
}, {
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "gold" : 0.5,
                "silver" : 0.5
            }
        ],
       

##Task 4

In [1]:
!pip install hmmlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 4.8 MB/s eta 0:00:00


In [8]:
'''Scenario:
A company, &quot;Smart Energy Solutions,&quot; specializes in providing renewable energy
solutions to households. They offer two types of energy products: solar panels and
wind turbines. The demand for these products can fluctuate based on various factors
such as weather conditions, government incentives, and consumer preferences. The
company wants to use a Hidden Markov Model (HMM) to predict the demand state of
each energy product based on historical sales data.
Task:
The company has collected sales data for a week for both types of energy products:
solar panels and wind turbines. Based on this data, they aim to train an HMM to predict
the demand state of each product (high-demand or low-demand) and determine the
transition and emission probabilities.'''

import numpy as np
from hmmlearn import hmm

solar_panels_sales = [20, 25, 30, 35, 30, 25, 20]
wind_turbines_sales = [10, 15, 20, 25, 20, 15, 10]

solar_panels_demand = np.array([1 if sales > np.mean(solar_panels_sales) else 0 for sales in solar_panels_sales])
wind_turbines_demand = np.array([1 if sales > np.mean(wind_turbines_sales) else 0 for sales in wind_turbines_sales])

X = np.column_stack([solar_panels_demand, wind_turbines_demand])

model = hmm.MultinomialHMM(n_components=2)
model.fit(X)

next_day_sales = [22, 28]
next_day_demand = np.array([1 if sales > np.mean(next_day_sales) else 0 for sales in next_day_sales])
predicted_demand = model.predict(next_day_demand.reshape(1, -1))
print("Predicted demand state for solar panels and wind turbines for the next day:", "High-demand" if predicted_demand[0] == 1 else "Low-demand")


https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Predicted demand state for solar panels and wind turbines for the next day: Low-demand


##Task 5

In [13]:
'''Scenario:
In a certain town, the weather can either be sunny or rainy. People in this town dress
differently based on the weather. On sunny days, they typically wear t-shirts, while on
rainy days, they wear jackets to stay dry.
Task:
We have a record of what people wear for a week but don&#39;t have direct observations of
the weather. Based on the clothing worn each day, use a Hidden Markov Model to infer
the most likely sequence of weather states (sunny or rainy) for the week.'''

import numpy as np
from hmmlearn import hmm

hidden_states = ['Sunny', 'Rainy']
observations = ['T-shirt', 'Jacket']

transition_probabilities = np.array([[0.8, 0.2], [0.3, 0.7]])

emission_probabilities = np.array([[0.9, 0.1], [0.2, 0.8]])

model = hmm.MultinomialHMM(n_components=len(hidden_states))
model.startprob_ = np.array([0.5, 0.5])
model.transmat_ = transition_probabilities
model.emissionprob_ = emission_probabilities

observations_sequence = ['T-shirt', 'Jacket', 'T-shirt', 'T-shirt', 'Jacket', 'Jacket', 'Jacket']
observations_indices = [observations.index(obs) for obs in observations_sequence]

model.fit(np.array(observations_indices).reshape(-1, 1))

inferred_states_indices = model.predict(np.array(observations_indices).reshape(-1, 1))
inferred_states = [hidden_states[state_index] for state_index in inferred_states_indices]


print("Inferred sequence of weather states for the week:")
print(inferred_states)


https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Inferred sequence of weather states for the week:
['Rainy', 'Sunny', 'Rainy', 'Sunny', 'Rainy', 'Sunny', 'Rainy']
